## 1. Set up

In [13]:
import pandas as pd
import os
import base64
import time
import pickle as p
import numpy as np
import matplotlib.pyplot as plt
import pandas.core.algorithms as algos
from sklearn.model_selection import train_test_split
from sklearn import metrics
from functools import reduce
import datetime
from dateutil.relativedelta import relativedelta
import logging
import matplotlib.style as style
import matplotlib
import matplotlib.lines as mlines
from matplotlib.ticker import FuncFormatter
style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')

In [14]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [15]:
import gc
gc.collect()

171

## 2. Data Loading & Preprocessing

### 2.1 TPA & Client

In [16]:
# Client & TPA

TPA = pd.read_excel(data_file_path + 'Copy_of_TPA_cash.xlsx',sheet_name='Sheet1')
Client = pd.read_excel(data_file_path + 'Client_info_20191118.xlsx',sheet_name='Sheet1')
# 3 Clients have duplicates records
# Client[Client['Client ID'].isin([1500,7601,2449])]
Client = Client.drop(Client.index[[0,1,497]])


Client = Client.merge(TPA,how='left',left_on='Client ID',right_on='HOST ID')
Client['TPA'] = Client['TPA_x']
Client.loc[Client['TPA_x'].isnull(),'TPA'] = Client.loc[Client['TPA_x'].isnull(),'TPA_y']
Client['TPA'] = Client['TPA'].apply(lambda x: x.upper() if pd.notnull(x) else np.nan)
Client.drop(['TPA_x','TPA_y','HOST ID'],axis=1,inplace = True)

In [17]:
# You might want to delete TPA, uncomment following line
# del TPA

### 2.2 Customer

In [18]:
Customer_old = pd.read_csv(data_file_path + 'Customer_Data_Request_20191121.csv')
Customer = pd.read_csv(data_file_path + 'Customer_Data_Request_20191217.csv')
# Append these 2 dataframes together
Customer = Customer_old.append(Customer,ignore_index = True)


# 2 Customer has duplicating records, we pick ClientID 2049 as their real Client but this is just a random choice
# Customer[Customer['Unique_Customer_ID']==100541023,100696306]
# Drop the duplicates customerID by index
Customer = Customer.drop(Customer.index[[72898,51730]])

# This line is self-explained
Customer = Customer.replace('.', np.nan)


# ===================== #
# Formatting conversion #
# ===================== #
def f(x):
    """
    This is a function that takes in any string format variable
    and return it as uppercase format.
    
    """
    if pd.notnull(x):
        return x.upper()
    else:
        return np.nan

Customer['Street1'] = Customer['Street1'].apply(lambda x: f(x))
Customer['Street2'] = Customer['Street2'].apply(lambda x: f(x))
Customer['City'] = Customer['City'].apply(lambda x: f(x))
Customer['Status'] = Customer['Status'].apply(lambda x: f(x))

# Convert numeric columns to float type
Customer_numeric_columns = ['Customer_Salary','Customer_Tenure']
Customer[Customer_numeric_columns] = Customer[Customer_numeric_columns].astype(float)

# Convert date format columns to datetime type
Customer_date_columns = ['Enrollment_Date','Term_Date']
Customer[Customer_date_columns] = Customer[Customer_date_columns].astype(str).apply(pd.to_datetime, format='%m/%d/%Y')

# Some data cleaning here, eliminate the customers with following cases
# Customer_Salary has NaN, 0 and '.' ~11%
# Assigned_Spending_Limit has 0 ~7%
# Customer_Tenure has NaN

# Customer = Customer[(Customer['Assigned_Spending_Limit'] != 0) & 
#                     (Customer['Customer_Salary'] != 0) & 
#                     (Customer['Customer_Salary'].notnull())&
#                     (Customer['Customer_Tenure'].notnull())]




# Merge Customer dataset with Client by Company ID/Client ID
Customer = Customer.merge(Client,how='left',left_on='Unique_Company_ID',right_on='Client ID')
Customer.head()


,Unique_Customer_ID,Unique_Company_ID,Enrollment_Date,Assigned_Spending_Limit,Customer_Tenure,Customer_Salary,Prior_Customer_Flag,Street1,Street2,City,State,Zip,Status,Term_Date,Active,Client ID,Account Name,Current Eligibles,Eligibles at time of Launch,Industry,NAICS Code,NAICS Description,Broker,Launch Date,Termination Date,Affiliate/Fed Gov?,CLIENT CODE,CLIENT NAME,CLIENT AR,Cash,TPA
0,100001115,1000,2014-02-24,6880.0,230.0,58229.0,Repeat Buyer,1471 N EAST ST,NaN,YORK,PA,17406,EMPLOYED,NaT,Y,1000.0,SOCIAL SECURITY ADMINISTRATION,60000.0,70000.0,Federal Government/Associations,923130.0,"Administration of Social, Human Resource and I...",PPC,2002-07-01,NaT,Fed Gov,SSA,SOCIAL SECURITY ADMINISTRATION,KELLY MENCHINGER,TPA sends cash on behalf of client,EAA
1,100001120,1000,2014-02-24,6520.0,362.0,109245.0,Repeat Buyer,94 BRENTWOOD LN,NaN,VALLEY STREAM,NY,11581,EMPLOYED,NaT,Y,1000.0,SOCIAL SECURITY ADMINISTRATION,60000.0,70000.0,Federal Government/Associations,923130.0,"Administration of Social, Human Resource and I...",PPC,2002-07-01,NaT,Fed Gov,SSA,SOCIAL SECURITY ADMINISTRATION,KELLY MENCHINGER,TPA sends cash on behalf of client,EAA
2,100001141,1000,2014-02-25,10290.0,337.0,127226.0,Repeat Buyer,103 KENILWORTH PARK DR,APT 2B,TOWSON,MD,21204,EMPLOYED,NaT,Y,1000.0,SOCIAL SECURITY ADMINISTRATION,60000.0,70000.0,Federal Government/Associations,923130.0,"Administration of Social, Human Resource and I...",PPC,2002-07-01,NaT,Fed Gov,SSA,SOCIAL SECURITY ADMINISTRATION,KELLY MENCHINGER,TPA sends cash on behalf of client,EAA
3,100001158,1000,2014-02-24,5190.0,419.0,49375.0,Repeat Buyer,1036 BRADDISH AVE,NaN,BALTIMORE,MD,21216,EMPLOYED,NaT,Y,1000.0,SOCIAL SECURITY ADMINISTRATION,60000.0,70000.0,Federal Government/Associations,923130.0,"Administration of Social, Human Resource and I...",PPC,2002-07-01,NaT,Fed Gov,SSA,SOCIAL SECURITY ADMINISTRATION,KELLY MENCHINGER,TPA sends cash on behalf of client,EAA
4,100001169,1000,2014-02-24,7500.0,448.0,69460.0,Repeat Buyer,3915 W ROGERS AVE,NaN,BALTIMORE,MD,21215,EMPLOYED,NaT,Y,1000.0,SOCIAL SECURITY ADMINISTRATION,60000.0,70000.0,Federal Government/Associations,923130.0,"Administration of Social, Human Resource and I...",PPC,2002-07-01,NaT,Fed Gov,SSA,SOCIAL SECURITY ADMINISTRATION,KELLY MENCHINGER,TPA sends cash on behalf of client,EAA


In [6]:
print('{:.0%} of Customers have Assigned_Spending_Limit == 0'.format(Customer[Customer['Assigned_Spending_Limit']==0].shape[0]/Customer.shape[0]))
print('{:.0%} of Customers have missing Customer_Salary'.format(Customer[Customer['Customer_Salary'].isnull()].shape[0]/Customer.shape[0]))
print('{:.2%} of Customers have Customer_Salary == 0'.format(Customer[Customer['Customer_Salary']==0].shape[0]/Customer.shape[0]))
print('{:.2%} of Customers have missing Customer_Tenure'.format(Customer[Customer['Customer_Tenure'].isnull()].shape[0]/Customer.shape[0]))

7% of Customers have Assigned_Spending_Limit == 0
11% of Customers have missing Customer_Salary
0.25% of Customers have Customer_Salary == 0
0.02% of Customers have missing Customer_Tenure


### 2.3 Performance Dataset

In [19]:
Perf_11 = pd.read_csv(data_file_path + 'Performance_Data_Request_20191217.csv')
Perf = pd.read_csv(data_file_path + 'Performance_Data_Request_20191204.csv')
# Deal with the column name mismatching issue before appending the 2 dataframes together
Perf_11.rename(columns={' Unique_Order_ID': 'Unique_Order_ID',
                        'Order_date': 'Order_Date'}, inplace=True)
Perf.rename(columns={'items_ordered': 'Items_Ordered',
                     'EMPSTATUS': 'Customer_Status'}, inplace=True)
# Append these 2 dataframes together
Perf = Perf.append(Perf_11, ignore_index=True)

# This line is self-explained
Perf = Perf.replace('.', np.nan)

# ===================== #
# Formatting conversion #
# ===================== #
# Convert numeric columns to float type
Perf_numeric_columns = ['Order_Amount', 'Product_Markup', 'Payments_Due', 'Current_Balance', 'Losses', 
                   'Spending_Limit_at_time_of_Order', 'Tenure_at_time_of_Order','Salary_at_time_of_Order']
Perf[Perf_numeric_columns] = Perf[Perf_numeric_columns].astype(float)

# Convert date format columns to datetime type
Perf_date_columns = ['Date_account_went_delinquent','Order_Date']
Perf[Perf_date_columns] = Perf[Perf_date_columns].astype(str).apply(pd.to_datetime,format = '%m/%d/%Y' )


# Make sure the data order is aligned with OrderID & Y-M
Perf['Year_and_Month'] = pd.to_datetime(Perf['Year_and_Month'].astype(str), format='%Y%m')
Perf = Perf.sort_values(['Unique_Order_ID', 'Year_and_Month'], ascending=[False, True])

# ======================== #
# New Columns Construction #
# ======================== #

# Create YQ and Year
Perf['YQ'] = pd.PeriodIndex(Perf['Year_and_Month'], freq='Q')
Perf['Year'] = Perf['YQ'].dt.year

# Create Payment_Order by Unique_Order_ID and Year_and_Month
Perf['Payment_Order'] = Perf.groupby('Unique_Order_ID')['Year_and_Month'].rank(method='first')

# Create Bad Flag - Treat 151+ and CO as BAD
Perf['Bad'] = 0
Perf.loc[Perf['Default_Status'].isin(['151+', 'Chargeoff']), 'Bad'] = 1

# Create a flag Ever_Bad to distinguish first time bad and non-first time bad
# If Ever_Bad == 1, this order is bad at the first time,
# If Ever_Bad >1, this order has been bad but not at the first time
# If Ever_Bad == 0, this order is good
Perf['Ever_Bad'] = Perf.groupby('Unique_Order_ID')['Bad'].transform(pd.Series.cumsum)

# Create 2os_loss
# Note that Orders with Default_Status == Chargeoff, the corresponding Current_Balance == 0, but Losses has valid value
# while Orders with Default_Status == 151+, the corresponding Current_Balance has valid value, but Losses == 0
# Thus, we use Losses for C/O orders and Current_Balance for 151+ orders to create 2os_loss
Perf['2os_loss'] = 0
Perf.loc[Perf['Default_Status'] == 'Chargeoff',
         '2os_loss'] = Perf.loc[Perf['Default_Status'] == 'Chargeoff', 'Losses']
Perf.loc[Perf['Default_Status'] == '151+',
         '2os_loss'] = Perf.loc[Perf['Default_Status'] == '151+', 'Current_Balance']



# Perf = Perf[(Perf['Order_Amount'].notnull()) & (Perf['Order_Amount'] != 0)]

# Merge Perf dataset with Customer by Unique_Customer_ID
Perf = Perf.merge(Customer, how='left', on='Unique_Customer_ID')

## 3. Start Constructing data

### 3.1 Generating Customer Level Data

In [14]:
# Create time_window mark for 18-month data selection later
Perf['time_window'] = Perf['Enrollment_Date'].dt.date + relativedelta(months=18)

In [15]:
def Perf_filter(df):
    """
    This function takes the Perf dataset and subgroup is with time window condition, 
    then it generates some of the customer-level variables that we expect, such as
    Num_of_Contracts,Total_CO_Balance,Total_Spend
    
    The output would be a customer-level dataframe, where the customers are enrolled in 2017-04 to 2018-04 time period
    """
    
    # Select customers that enrolled in the past 18-30 months (mature)
    df_sub = df[(df['Enrollment_Date'] <= pd.to_datetime('2018-04'))
                & (df['Enrollment_Date'] >= pd.to_datetime('2017-04'))]
    # Get the latest order records
    df_sub = df_sub.loc[df_sub.groupby('Unique_Order_ID')[
        'Year_and_Month'].idxmax()]
    # create customer level data with "# of orders" and "Bad Flag"
    df_sub = df_sub.groupby('Unique_Customer_ID').agg(
        {'Unique_Order_ID': 'count', 'Ever_Bad': 'sum', '2os_loss': 'sum', 'Order_Amount': 'sum'}).reset_index()
    # The definition of Bad here is whether a customer has at least 1 order goes bad
    df_sub['Bad'] = df_sub['Ever_Bad'] >= 1

    df_sub.rename(columns={'Unique_Order_ID': 'Num_of_Contracts',
                           '2os_loss': 'Total_CO_Balance', 'Order_Amount': 'Total_Spend'}, inplace=True)

    return df_sub

In [16]:
tmp = Perf_filter(Perf)

In [17]:
tmp.shape

(73729, 6)

In [18]:
tmp.head()

,Unique_Customer_ID,Num_of_Contracts,Ever_Bad,Total_CO_Balance,Total_Spend,Bad
0,302641212,1,0,0.0,1889.29,False
1,302641217,1,0,0.0,304.34,False
2,302641220,3,0,0.0,4516.09,False
3,302641222,15,0,0.0,7546.45,False
4,302641226,13,0,0.0,8152.89,False


In [19]:
# Let's just call our first merged dataset df..., merge tmp with Customer, we'll have `Customer_Tenure`, 
# `Customer_Tenure` and `Assigned_Spending_Limit`...
df = tmp[['Unique_Customer_ID', 'Bad', 'Num_of_Contracts','Total_CO_Balance','Total_Spend']].merge(
    Customer, how='left', on='Unique_Customer_ID')

In [20]:
df.shape

(73729, 18)

In [21]:
df.head()

,Unique_Customer_ID,Bad,Num_of_Contracts,Total_CO_Balance,Total_Spend,Unique_Company_ID,Enrollment_Date,Assigned_Spending_Limit,Customer_Tenure,Customer_Salary,Prior_Customer_Flag,Street1,Street2,City,State,Zip,Status,Term_Date
0,302641212,False,1,0.0,1889.29,2183,2017-04-01,7260.0,317.0,47196.00,First Time Byer,214 S KESSING ST,NaN,PORTERVILLE,CA,93257,Employed,NaN
1,302641217,False,1,0.0,304.34,2302,2017-04-01,500.0,44.0,11804.00,First Time Byer,6580 MORET DR S,NaN,JACKSONVILLE,FL,32244,Direct Bill,NaN
2,302641220,False,3,0.0,4516.09,2250,2017-04-01,2940.0,34.0,34037.38,Repeat Buyer,2967 GORDON RD,NaN,ROCK HILL,SC,29732,Employed,NaN
3,302641222,False,15,0.0,7546.45,2425,2017-04-01,3065.0,43.0,35495.09,Repeat Buyer,9231 S CYPRESS CIR,NaN,MIRAMAR,FL,33025,Employed,NaN
4,302641226,False,13,0.0,8152.89,2464,2017-04-01,1500.0,33.0,55823.00,Repeat Buyer,2266 S HARWOOD AVE,NaN,UPPER DARBY,PA,19082,Direct Bill,NaN


### 3.2 Generating `Client Risk`
- The reason why a fixed time window is used here is it's hard to track each customer's enrollment date to get a cutomer-level Client Risk, so instead of generating a unique Client Risk for each customer, we use an approximate method to get a same Client Risk for all customers from that specific Client.

In [22]:
Perf_p12m = Perf[(Perf['Year_and_Month'] <= pd.to_datetime('2017-04'))              
                & (Perf['Year_and_Month'] >= pd.to_datetime('2016-04'))]

In [23]:
# Get Total CO Balance per Client
client_loss = Perf_p12m[(Perf_p12m['Bad'] == 1) & (
    Perf_p12m['Ever_Bad'] <= 1)].groupby('Unique_Company_ID')['2os_loss'].sum().reset_index()

In [24]:
# Get Total Spend per Client
client_order_vol = Perf_p12m.groupby(['Unique_Company_ID', 'Unique_Order_ID']).head(
    1).groupby('Unique_Company_ID')['Order_Amount'].sum().reset_index()

In [25]:
# I really don't know how to name this dataset
client_loss_m = client_loss.merge(client_order_vol, how='right',on='Unique_Company_ID')

In [26]:
client_loss_m.head()

,Unique_Company_ID,2os_loss,Order_Amount
0,1000.0,216543.28,11203596.54
1,1200.0,17430.36,710638.32
2,1301.0,60.60,6820.50
3,1302.0,538.91,50913.44
4,1500.0,3379.07,1679927.63


In [27]:
client_loss_m['Client_Risk'] = client_loss_m['2os_loss']/client_loss_m['Order_Amount']

In [28]:
# Some client may have no loss where 2os_loss is NaN, resulting loss_rate == NaN, fill these with 0
client_loss_m =  client_loss_m.fillna(0)

In [29]:
# Now merge the Clint risk to our first merged dataset df
df = df.merge(client_loss_m[['Unique_Company_ID','Client_Risk']], how = 'left',on= 'Unique_Company_ID')

### 3.3 Final Steps..

In [30]:
# Customers with missing `Client_Risk` means their company don't have any orders prior(12m) to their enrollment
# Some Salary number seems to be ridiculous, treat them as outliers
final = df[(df['Client_Risk'].notnull())&(df['Customer_Salary']>2000) &(df['Customer_Salary']!=9999999.99)]

In [31]:
# See the summary of important features
final[['Customer_Tenure','Customer_Salary','Assigned_Spending_Limit','Client_Risk','Num_of_Contracts','Total_CO_Balance','Total_Spend']].describe()

,Customer_Tenure,Customer_Salary,Assigned_Spending_Limit,Client_Risk,Num_of_Contracts,Total_CO_Balance,Total_Spend
count,72031.000000,72031.000000,72031.000000,72031.000000,72031.000000,72031.000000,72031.000000
mean,79.702989,41366.802059,3658.995904,0.027663,4.273466,234.993017,3089.612642
std,71.252333,19453.695191,2228.914229,0.026077,5.094489,655.422507,3153.956408
min,1.000000,2308.000000,400.000000,0.000000,1.000000,0.000000,11.380000
25%,38.000000,29827.200000,1875.000000,0.002049,1.000000,0.000000,1018.835000
50%,50.000000,35741.000000,3260.000000,0.033255,3.000000,0.000000,2058.390000
75%,90.000000,47819.200000,4930.000000,0.035308,5.000000,0.000000,4016.765000
max,634.000000,762720.000000,13160.000000,0.316066,103.000000,9959.360000,41403.970000


In [176]:
# final.isnull().sum(axis=0)

## Big Time: Output the final dataset

In [ ]:
# Save to local
# final.to_csv([FILE PATH/FILENAME] +'.csv')

In [ ]:
# Save to S3 Bucket
import s3fs

s3 = s3fs.S3FileSystem(anon=False)

# Use 'w' for py3, 'wb' for py2
with s3.open('power-purchasing/Customer_level_data.csv','w') as f:
    final.to_csv(f)

In [ ]:
#----------------------------------------------- END ------------------------------------------------------#

In [32]:
final.shape

(72031, 19)

In [33]:
final.head()

,Unique_Customer_ID,Bad,Num_of_Contracts,Total_CO_Balance,Total_Spend,Unique_Company_ID,Enrollment_Date,Assigned_Spending_Limit,Customer_Tenure,Customer_Salary,Prior_Customer_Flag,Street1,Street2,City,State,Zip,Status,Term_Date,Client_Risk
0,302641212,False,1,0.0,1889.29,2183,2017-04-01,7260.0,317.0,47196.00,First Time Byer,214 S KESSING ST,NaN,PORTERVILLE,CA,93257,Employed,NaN,0.021343
1,302641217,False,1,0.0,304.34,2302,2017-04-01,500.0,44.0,11804.00,First Time Byer,6580 MORET DR S,NaN,JACKSONVILLE,FL,32244,Direct Bill,NaN,0.033438
2,302641220,False,3,0.0,4516.09,2250,2017-04-01,2940.0,34.0,34037.38,Repeat Buyer,2967 GORDON RD,NaN,ROCK HILL,SC,29732,Employed,NaN,0.040143
3,302641222,False,15,0.0,7546.45,2425,2017-04-01,3065.0,43.0,35495.09,Repeat Buyer,9231 S CYPRESS CIR,NaN,MIRAMAR,FL,33025,Employed,NaN,0.068904
4,302641226,False,13,0.0,8152.89,2464,2017-04-01,1500.0,33.0,55823.00,Repeat Buyer,2266 S HARWOOD AVE,NaN,UPPER DARBY,PA,19082,Direct Bill,NaN,0.002049


In [34]:
def view(var,df):
    print('Spending Limit by {}'.format(var))
    df[var+'_bin'] = pd.qcut(df[var], q=4)
    return df.groupby(var+'_bin').agg({'Assigned_Spending_Limit':'mean','Unique_Customer_ID':'count'}).round(2)

In [35]:
view('Customer_Salary',final)

Spending Limit by Customer_Salary


,Assigned_Spending_Limit,Unique_Customer_ID
Customer_Salary_bin,,
"(2307.999, 29827.2]",1605.42,18010
"(29827.2, 35741.0]",2722.55,18006
"(35741.0, 47819.2]",4046.47,18009
"(47819.2, 762720.0]",6261.94,18006


In [120]:
view('Customer_Tenure',final)

Spending Limit by Customer_Tenure


,Assigned_Spending_Limit,Unique_Customer_ID
Customer_Tenure_bin,,
"(0.999, 38.0]",2953.69,19592
"(38.0, 50.0]",3025.98,16588
"(50.0, 90.0]",3814.54,17910
"(90.0, 634.0]",4859.21,17941


In [121]:
view('loss_rate',final)

Spending Limit by loss_rate


,Assigned_Spending_Limit,Unique_Customer_ID
loss_rate_bin,,
"(-0.001, 0.00205]",3187.45,18539
"(0.00205, 0.0333]",4359.32,18094
"(0.0333, 0.0353]",3662.20,18628
"(0.0353, 0.316]",3421.10,16770


In [122]:
def risk_view(var,df):
    print('Bad Rate by {}'.format(var))
    df[var+'_bin'] = pd.qcut(df[var], q=4)
    return df.groupby(var+'_bin')['Bad'].sum().div(df.groupby(var+'_bin')['Unique_Customer_ID'].size())

In [123]:
risk_view('Customer_Salary',final)

Bad Rate by Customer_Salary


Customer_Salary_bin
(2307.999, 29827.2]    0.362132
(29827.2, 35741.0]     0.273131
(35741.0, 47819.2]     0.177578
(47819.2, 762720.0]    0.136621
dtype: float64

In [124]:
risk_view('Customer_Tenure',final)

Bad Rate by Customer_Tenure


Customer_Tenure_bin
(0.999, 38.0]    0.281748
(38.0, 50.0]     0.312696
(50.0, 90.0]     0.227192
(90.0, 634.0]    0.129424
dtype: float64

In [173]:
risk_view('Client_Risk',final)

Bad Rate by Client_Risk


Client_Risk_bin
(-0.001, 0.00205]    0.222018
(0.00205, 0.0333]    0.188792
(0.0333, 0.0353]     0.245491
(0.0353, 0.316]      0.297734
dtype: float64

In [174]:
print('Bad Rate vs Spending Limit')
final['Spending_Limit_bin'] = pd.qcut(final['Assigned_Spending_Limit'], q=6)
final.groupby('Spending_Limit_bin')['Bad'].sum().div(final.groupby('Spending_Limit_bin')['Unique_Customer_ID'].size())

Bad Rate vs Spending Limit


Spending_Limit_bin
(399.999, 1445.0]    0.365557
(1445.0, 2330.0]     0.312217
(2330.0, 3260.0]     0.267938
(3260.0, 4335.0]     0.212948
(4335.0, 5620.0]     0.154103
(5620.0, 13160.0]    0.108509
dtype: float64